In [2]:
import pandas as pd
df = pd.read_csv('../../data/preprocessed_v2/clear_augmentation_v2.csv')
df

,ID,text,target
0,ynat-v1_train_00000,정의구현사제단 시국미사 미사용기간 단기간에 종결될 수 있다,4
1,ynat-v1_train_00002,"김정일, 자주, 통일, 새, 열, 나가야, 보수",2
2,ynat-v1_train_00004,대통령 선거를 앞두고 정치권 비방 댓글 단에 대한 감시가 강화되고 있다.,6
3,ynat-v1_train_00006,광주-기아챔피언스필드에서 열릴 예정이던 롯데-KIA 경기가 우천으로 취소됐다.,1
4,ynat-v1_train_00007,아포짓 스파이커 알렉스와 레오나르도 가빈이 이끄는 한국전력 빅스톰이 우리캐피탈 위비...,4
...,...,...,...
6049,augmentation,5 . 게임 산업의 괄목할 성장과 사회적 영향,4
6050,augmentation,이제 특이하고 매적인 문화 뉴스 제 목 6개를 만들어볼 차례야,0
6051,augmentation,1 . 축구 월드컵 예선 한 국 팀 선전 분석,1
6052,augmentation,2 . 신흥 강호 로 부상 하고 있는 한국 배구 리그,1


In [3]:
df['text_len'] = df['text'].apply(lambda x: len(x.split()))
df['text_len'].describe()

count    6054.000000
mean        7.405517
std         2.689058
min         1.000000
25%         6.000000
50%         7.000000
75%         8.000000
max        29.000000
Name: text_len, dtype: float64

In [5]:
from transformers import AutoTokenizer

model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

df['token_len'] = df['text'].apply(lambda x: len(tokenizer(x)['input_ids']))
df['token_len'].describe()

count    6054.000000
mean       15.513214
std         7.213673
min         4.000000
25%        11.000000
50%        14.000000
75%        17.000000
max        70.000000
Name: token_len, dtype: float64

In [4]:
import pandas as pd

df = pd.read_csv('../../data/raw/train.csv')
df

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2
2797,ynat-v1_train_02797,텔레그램+한D 등h亞서 2시간H다운…C버T정gf39종!2보,4
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [4]:
df['text_len'] = df['text'].apply(len)
df['text_len'].describe()

count    2800.000000
mean       27.192143
std         4.957505
min         8.000000
25%        25.000000
50%        28.000000
75%        31.000000
max        41.000000
Name: text_len, dtype: float64

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("SEOKDONG/llama3.0_korean_v1.0_sft")

# check token length
def get_token_length(x):
    return len(tokenizer(x)['input_ids'])

df['token_length'] = df['text'].apply(get_token_length)

In [ ]:
df['token_length'].describe()

count    2800.000000
mean       22.198571
std         4.471960
min         7.000000
25%        20.000000
50%        23.000000
75%        25.000000
max        37.000000
Name: token_length, dtype: float64

In [1]:
prompt = (
    "다음은 '*'로 가려진 한국어 뉴스 기사 제목이야.\n"
    "주어진 뉴스 기사 제목 중 '*'로 가려져 있지 않는 부분을 활용하여 숙련된 한국 뉴스 기자가 쓸 법한 한국어 뉴스 기사 제목을 반말로 생성해.\n"
)
prompt   

"다음은 '*'로 가려진 한국어 뉴스 기사 제목이야.\n주어진 뉴스 기사 제목 중 '*'로 가려져 있지 않는 부분을 활용하여 숙련된 한국 뉴스 기자가 쓸 법한 한국어 뉴스 기사 제목을 반말로 생성해.\n"

In [2]:
prompt = """다음은 한국어 뉴스 헤드라인을 작성하는 과제야. 너는 숙련된 한국 뉴스 기자처럼 생각하며 최신 뉴스 트렌드와 보도 방식을 반영해 짧고 간결한 헤드라인을 작성해야 해.

작성 지침:
1. 주제와 세부사항은 너의 상상력에 맡긴다. 정치, 경제, 사회, 문화 등 다양한 분야를 다룰 수 있어.
2. 헤드라인은 15자에서 30자 사이로 작성해.
3. 독자의 흥미를 끌 수 있도록 매력적이고 주목할 만한 내용을 반영해.
4. 중립적이고 사실 기반의 어조를 유지해.

예시:
- 헤드라인: '국내 IT기업, 혁신 기술 발표'
- 헤드라인: '서울, 대기질 개선 프로젝트 착수'
- 헤드라인: '여자농구 신한은행 팀 최소 득점 기록 34점 불명예'
- 헤드라인: '메시·호날두 UEFA 올해의 팀에 선정…EPL 선수 제로'
- 헤드라인: '여당, 신규 정책으로 국민 지지율 상승'

이제 자유롭게 주제와 세부사항을 설정하고 헤드라인을 작성해봐.
"""

prompt

"다음은 한국어 뉴스 헤드라인을 작성하는 과제야. 너는 숙련된 한국 뉴스 기자처럼 생각하며 최신 뉴스 트렌드와 보도 방식을 반영해 짧고 간결한 헤드라인을 작성해야 해.\n\n작성 지침:\n1. 주제와 세부사항은 너의 상상력에 맡긴다. 정치, 경제, 사회, 문화 등 다양한 분야를 다룰 수 있어.\n2. 헤드라인은 15자에서 30자 사이로 작성해.\n3. 독자의 흥미를 끌 수 있도록 매력적이고 주목할 만한 내용을 반영해.\n4. 중립적이고 사실 기반의 어조를 유지해.\n\n예시:\n- 헤드라인: '국내 IT기업, 혁신 기술 발표'\n- 헤드라인: '서울, 대기질 개선 프로젝트 착수'\n- 헤드라인: '여자농구 신한은행 팀 최소 득점 기록 34점 불명예'\n- 헤드라인: '메시·호날두 UEFA 올해의 팀에 선정…EPL 선수 제로'\n- 헤드라인: '여당, 신규 정책으로 국민 지지율 상승'\n\n이제 자유롭게 주제와 세부사항을 설정하고 헤드라인을 작성해봐.\n"

In [12]:
import pandas as pd

# 리스트 정의
l = ['예시1', '예시2', '예시3', '예시4', '예시5']

# 새로운 DataFrame 생성
df = pd.DataFrame({
    'ID': ['augmentation'] * len(l),  # ID 열을 'augmentation'으로 채움
    'text': l,  # text 열에 리스트 l 삽입
    'target': [-1] * len(l)  # target 열을 -1로 채움
})

# 결과 출력
x = df.at[0, 'ID']
type(x)

str

In [41]:
import re


text = """"<|START_OF_TURN_TOKEN|><|USER_TOKEN|>다음은 한국어 뉴스 헤드라인을 작성하는 과제야. 너는 숙련된 한국 뉴스 기자처럼 생각하며 최신 뉴스 트렌드와 보도 방식을 반영해 짧고 간결한 헤드라인을 작성해야 해.

작성 지침:
1. 주제와 세부사항은 너의 상상력에 맡긴다. 정치, 경제, 사회, 문화 등 다양한 분야를 다룰 수 있어.
2. 헤드라인은 15자에서 30자 사이로 작성해.
3. 독자의 흥미를 끌 수 있도록 매력적이고 주목할 만한 내용을 반영해.
4. 중립적이고 사실 기반의 어조를 유지해.
5. ""- 헤드라인"" 이라는 문구를 포함해 헤드라인을 작성해.

예시:
- 헤드라인: '국내 IT기업, 혁신 기술 발표'
- 헤드라인: '서울, 대기질 개선 프로젝트 착수'
- 헤드라인: '여자농구 신한은행 팀 최소 득점 기록 34점 불명예'
- 헤드라인: '메시·호날두 UEFA 올해의 팀에 선정…EPL 선수 제로'
- 헤드라인: '여당, 신규 정책으로 국민 지지율 상승'

이제 자유롭게 주제와 세부사항을 설정하고 헤드라인을 작성해봐.<|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>다음은 제가 작성한 한국어 خبر르라인입니다:

1.- 헤د라인 : '정부, 탈탄소 정책 추진...2050년 탄소중립 목표' 
2.- ヘ르아라인： 'K-팝 열풍, 세계 음악 시장 지형 바꾼다'  
3.- 赫드ライン：'서울시, 도심 속 공유주택 확대 추진'   
4.- 頭드 라인： 'AI 기술, 의료 분야 혁신을"
"""

# 헤드라인 추출
headlines = text.split('헤드라인을 작성해봐.')[1]

# 결과 출력
print(headlines)

<|START_OF_TURN_TOKEN|><|CHATBOT_TOKEN|>다음은 제가 작성한 한국어 خبر르라인입니다:

1.- 헤د라인 : '정부, 탈탄소 정책 추진...2050년 탄소중립 목표' 
2.- ヘ르아라인： 'K-팝 열풍, 세계 음악 시장 지형 바꾼다'  
3.- 赫드ライン：'서울시, 도심 속 공유주택 확대 추진'   
4.- 頭드 라인： 'AI 기술, 의료 분야 혁신을"



In [29]:
s = set()
s.add(134)
s.add(244)
s

{134, 244}

In [38]:
df = pd.DataFrame({
    'ID': ['start'],
    'text': ['start'],
    'target': ['start'],
})
df

,ID,text,target
0,start,start,start
